In [22]:
from src.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from src.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.Base.Evaluation.Evaluator import EvaluatorHoldout
from src.Utils.load_URM import load_URM
from src.Utils.load_ICM import load_ICM
import numpy as np

URM_all = load_URM("in/data_train.csv")
ICM_all = load_ICM("in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [23]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])


In [24]:
MAP=[]


Similarity_1 = EASE_R_Recommender(URM_train=URM_train, sparse_threshold_quota=1.0)

Similarity_1.fit(topK=100)

Similarity_1.save_model(
    folder_path='C:/Users/ALEXDRUSO/Desktop/RecSysChallenge2020/models/',
    file_name=EASE_R_Recommender.RECOMMENDER_NAME+'.zip'
)

Similarity_1.load_model(
    folder_path='C:/Users/ALEXDRUSO/Desktop/RecSysChallenge2020/models/',
    file_name=EASE_R_Recommender.RECOMMENDER_NAME+'.zip'
)
Similarity_1 = Similarity_1.W_sparse
Similarity_2 = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=ICM_all)
Similarity_2.fit(
    topK=100,
    shrink=10,
    similarity='jaccard'
)
Similarity_2 = Similarity_2.W_sparse

for topK in [10, 50, 100, 200, 500]:
    for alpha in np.linspace(start=0.0,stop=1.0,num=10):
        recommender = ItemKNNSimilarityHybridRecommender(URM_train=URM_train, Similarity_1=Similarity_1, Similarity_2=Similarity_2)
        recommender.fit(topK=topK, alpha=alpha)

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

        MAP.append(('topK={}'.format(topK),'alpha={}'.format(alpha),result_dict[10]["MAP"]))


EASE_R_Recommender: URM Detected 69 (0.87 %) cold users.
EASE_R_Recommender: URM Detected 2430 (9.36 %) cold items.
EASE_R_Recommender: Fitting model... 
Similarity column 25975 ( 100 % ), 9164.38 column/sec, elapsed time 0.05 min
EASE_R_Recommender: Fitting model... done in 11.56 min
EASE_R_Recommender: Detected model matrix to be sparse, changing format.
EASE_R_Recommender: Saving model in file 'C:/Users/ALEXDRUSO/Desktop/RecSysChallenge2020/models/EASE_R_Recommender.zip'
EASE_R_Recommender: Saving complete
EASE_R_Recommender: Loading model from file 'C:/Users/ALEXDRUSO/Desktop/RecSysChallenge2020/models/EASE_R_Recommender.zip'
EASE_R_Recommender: Loading complete
ItemKNNCBFRecommender: URM Detected 69 (0.87 %) cold users.
ItemKNNCBFRecommender: URM Detected 2430 (9.36 %) cold items.
Similarity column 25975 ( 100 % ), 4190.27 column/sec, elapsed time 0.10 min
ItemKNNSimilarityHybridRecommender: URM Detected 69 (0.87 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 2430

In [25]:
max_tuple = max(MAP, key= lambda x: x[2])
print('The best combination for the hybrid recommender is ({},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2]))

The best combination for the hybrid recommender is (topK=100,alpha=1.0), with a MAP = 0.0527988025506705
